## A Chatbot using GPT and a Database
This allows multiple chatbot types (e.g. a health coach and a learning assistant) to be created. Multiple chatbot instances can be created per chatbot type (e.g. for User X and User Y). Both, type and instance are stored and referenced with an ID (e.g. with a UUID) in the database.

This can support the deployment of chatbots in a web backend (state-less). For example, the UUIDs of the type and instance can be read as URL parameters from a URL that users have received from you.

A chatbot is created with the following arguments.
- database_file: File of SQLite (in Folder data/)
- type_id: Reference to chatbot type
- instance_id: Reference to chatbot instance (typically one per user - however, may also be shared by multiple users)
- type_role: Role of chatbot type (will be turned into a first prompt with role:system)
- instance_context: Context of chatbot instance (will be turned into a second prompt with role:system)
- instance_starter: Will be used to generate an initial message to the user (will be turned into a final prompt with role:system)

#### OpenAI API Key and Model

Rename the file chatbot/openai_template.py to openai.py and set the keys
    OPENAI_KEY = "your OpenAI API key"\
    OPENAI_MODEL = "gpt-3.5-turbo-16k"\

In [ ]:
%pip install openai

#### Import chatbot library

In [1]:
from chatbot.chatbot import Chatbot

#### Create a chatbot "Puzzle Workshop" for user X

URL to be handed out to the user: If you are following the instructions to deploy your chatbot(s) to pythonanywhere, this is the URL to access your chatbot.

##### Generic URL
https://[your pythonanywhere user name].pythonanywhere.com/[type id]/[user_id]/chat
##### For Example
https://monkey23.pythonanywhere.com/053e97a0-6a91-4589-8602-340aa47b6376/7515865e-4097-4dd7-9567-d3c7a4c1ed07/chat

#### Prompt Engineering
Before we create our bot and store it in the dabase, we can edit the prompts and customise our bot or just use the once as prepared below.

##### type_role
Describes the general role of the chatbot.
    - What kind of persona should it enact/imitate. 
    - What is the context it acts in
    - What is the goal of the conversation.
    
This prompt will be turned into a first prompt with role:system

In [6]:
type_role = """
Puzzle-Workshop

Du bist ein Gastgeber eines Puzzle-Lösungs-Workshops. Führe eine Unterhaltung mit einer Person, während sie die Rätsel versucht.

Regeln:
- Bleibe beim Thema.
- Gib niemals die Antworten preis.
- Lobe richtige Antworten und weise auf falsche hin.

Rätsel:
- Game & In-App-Kauf: CHF 33. Game: CHF 30 mehr als In-App-Kauf. Preis vom Game?
- 3 Ärzte brauchen 3 Minuten um 3 Patienten zu impfen. Wie lange für 7 Ärzte, 7 Patienten?
- Handyakku halbiert sich jährlich. 1 Stunde im Jahr 7. Wann war es doppelt so viel?

Lösungen (zur Überprüfung, nicht zur Offenlegung):
1. CHF 31.50 (CHF 31.50 und CHF 1.50 ergeben CHF 33, während der Unterschied CHF 30 beträgt).
2. 3 Minuten (jeder Arzt braucht 3 Minten, um einen Patient zu impfen, daher benötigen 7 Ärzte 3 Minuten, um 7 Patienten zu impfen).
3. Jahr 6 (Akkulaufzeit betrug 2 Stunden, halbiert auf 1 Stunde im Jahr 7).

Interaktionsmöglichkeiten:
1. Workshop-Info
2. Ein Rätsel erhalten
3. Hilfe nach 2 falschen Versuchen.
4. Leistungsbeurteilung wenn alle Rätsel gelöst.
"""

##### instance_context 
Defines the context, in which the chatbot acts.
- Define how the answers of the chatbot should be formatted (here we added some html for the frontend)
- Define how the chatbot should be answering to the user


It will be turned into a second prompt with role:system.

In [ ]:
instance_context = """
<p>Bei Antworten:</p>
<ol>
    <li>Emojis immer dann einbinden, wenn es passt. 😊</li>
    <li>Achte darauf, dass die Antworten vollständig und präzis sind, ohne mit einem Doppelpunkt oder mit '... folgendes:' zu enden.</li>
    <li>Verwende <b>&lt;ol&gt;/&lt;ul&gt;</b> mit <b>&lt;li&gt;</b>, um Informationen in Listenform zu präsentieren, selbst wenn sie kurz sind.</li>
    <li>Wenn es die Möglichkeit gibt, mehr als eine Information oder ein Feedback zu geben, teile sie in mehrere <b>&lt;p&gt;</b>-Elemente auf, um eine bessere Klarheit zu gewährleisten.</li>
    <li>Formatiere alle Antworten immer mit gültigem HTML: z.B. <b>&lt;p&gt;</b> für Absätze, <b>&lt;ul&gt;/&lt;ol&gt;</b> mit <b>&lt;li&gt;</b> für Listen und <b>&lt;b&gt;</b> zur Hervorhebung.</li>
    <li>Halte einen nihilistischen humorvollen Ton bei. Halte es kurz, aber opfere nicht die Klarheit für Kürze.</li>
</ol>
"""

##### instance_starter

Will be used to generate an initial message to the user
- what should the chatbot start the conversation with
- ideally describe how the conversation shall continue after --> creates a more natural opening.

Will be turned into a final prompt with role:system

In [1]:

instance_starter = """
Jetzt, frage nach dem Namen und einem persönlichen Detail (z.B. Hobby, Beruf, Lebenserfahrung).
Verwende diese im geschlechtsneutralem Gespräch in Du-Form.
Sobald ein Name und persönliches Detail bekannt ist, zeige eine Liste von Optionen.
"""

#### Creating multiple instances of chatbot "Coach"
In the following, we can either create a bot for one user or for n users. Each instance has it's own prompts (instance context and starter) that will be appended to the type prompts. Most importantly, each instance has its own chat history.

In [ ]:
import uuid
import time

If you want to create more than one instance, change `number_of_instances`

In [ ]:
# Amount of instances to be created
number_of_instances = 1

# Change the following to a list of hardcoded instance IDs if you want to use existing users.
user_ids = [str(uuid.uuid4()) for _ in range(number_of_instances)]

c  = 0 # counter for successful requests, don't change
error_c = 0 # counter for failed requests, don't change
for user_id in user_ids:
    bot = Chatbot(
        database_file="database/chatbot.db", 
        type_id="4b9e7b25-08d7-4031-b485-4741ecd48dd1",
        user_id=user_id,
        type_name = "PuzzleBot",
        type_role = type_role,
        instance_context=instance_context,
        instance_starter=instance_starter
    )
    try:
        # each bot should have a first message to the user
        print(bot.start())
    except:
        error_c += 1
        continue
    c+=1
    time.sleep(15) #openai seems to produce more errors if we send the requests too fast.
    
print("successful: {}, failed: {}".format(c, error_c))


#### Obtain URLs of all instances of a type
We need one instance of that type and can then use the type_instances() function to retrieve all of instance ids. Using these instance ids we can then create URLs such as for pythonanywhere environment.

- Enter your pythonanywhere username in the placeholder below.
- Change the type_id if you changed it above.

In [ ]:
pythonanywhere_username = "<ENTER YOUR PYTHONANYWHERE USERNAME HERE>"
type_id = "44b9e7b25-08d7-4031-b485-4741ecd48dd1"
bot = Chatbot(
    database_file="database/chatbot.db", 
    type_id=type_id,
    user_id=user_ids[0]
)

for user_id in bot.type_instances():
    print("https://{}.pythonanywhere.com/{}/{}/chat".format(pythonanywhere_username, type_id, user_id))